**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [8]:
class Environment(object):
    def __init__(self):
        pass
        
    def act(self, act):
        
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        "self.act"
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [9]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

**Explanation of the function act:**<br>
The function act determines the action to perform by the agent considering the current state and the previous steps used. During the training phase, we add a possibility to do a random state (which means that the agent will perform a step without taking into consideration the past one) so the action will not always promote similar actions. <br>

**Explanation of the parameter epsilon:**<br>
The parameter epsilon sets the tendancy of the agent to act randomly at each step. A value of epsilon close to 0 will make the agent only act on what he has learned and a value of epsilon close to 1 will correspond to a random action at each step. This parameter is essential because if the agent acts only on what it has learned, it will promote too much certain actions that have higher rewards and prevent other exploration to be possible.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [10]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [11]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position contains all the possible positions that can be taken by the animal. The value of a cell is -1 if the animal can't go there, 1 if it is already there, and 0 if it can go to this cell. The board represents the reward associated with each position if the rat goes there.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [12]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # we only have to set a parameter a to get the random actions
        a = np.random.randint(0, self.n_action, size=1)[0]
        pass

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [13]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    # for each epoch we reset the whole environment    
    for e in range(epochs):
        state = env.reset()

        
    # The variable game_over allows us to stop the game if it is over
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent can perform an action (here we perform it without the parameter epsilon, we are in the test phase)
            action = agent.act(state,train=False)

            # we need the previous state, the new state, the reward to learn. We also need the variable game_over
            
            prev_state = state
            state, reward, game_over = env.act(action)

            # If the reward is positive, we add the reward to the win cases
            if reward > 0:
                win = win + reward
            # otherwise we substract it from the loss (loss=absolute value of negative rewards)
            if reward < 0:
                lose = lose -reward

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [16]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 2.0/2.0. Average score (0.0)
Win/lose count 2.5/1.0. Average score (0.75)
Win/lose count 1.0/4.0. Average score (-0.5)
Win/lose count 1.0/2.0. Average score (-0.625)
Win/lose count 2.0/2.0. Average score (-0.5)
Win/lose count 1.5/3.0. Average score (-0.6666666666666666)
Win/lose count 0.5/1.0. Average score (-0.6428571428571429)
Win/lose count 3.5/2.0. Average score (-0.375)
Win/lose count 1.0/4.0. Average score (-0.6666666666666666)
Win/lose count 1.0/2.0. Average score (-0.7)
Win/lose count 3.5/4.0. Average score (-0.6818181818181818)
Final score: -0.6818181818181818


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

Ecrire l'ojective function
Allows to enforce the previous equality, if our neural network perfectly converge this equality must be true. Using this loss function we try to enforce our learned Q-value to verify the Bellman equation.

**(1)** We saw in class that : <br>
\begin{equation*}
Q^{\pi}(s,a)=E[\sum\limits_{k=0}^{\infty} \gamma^k r_{t+k} | s_{t}=s,a_{t}=a]
\end{equation*}
So we have
\begin{equation*}
Q^{\pi}(s,a)=E[r(s,a)+\sum\limits_{k=1}^{\infty} \gamma^k r_{t+k} | s_{t}=s,a_{t}=a]
\end{equation*}
\begin{equation*}
Q^{\pi}(s,a)=r(s,a)+\gamma E[E[\sum\limits_{k=0}^{\infty} \gamma^k r_{t+k+1} | s_{t+1}=s',a_{t+1}=a']| s_{t}=s,a_{t}=a]
\end{equation*}
\begin{equation*}
Q^{\pi}(s,a)=r(s,a)+\gamma \sum\limits_{s',a'}p(s_{t+1}=s',a_{t+1}=a'|s_{t}=s,a_{t}=a) E[\sum\limits_{k=0}^{\infty} \gamma^k r_{t+k+1} | s_{t+1}=s',a_{t+1}=a']
\end{equation*}
and since we have that $E_{(s',a')\sim p(.|s,a)}[r(s,a)]=r(s,a)$

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] (1)
\end{equation*}

**2**
\begin{equation*}
Q^{*}(s,a)=max_{\pi}  (E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')])
\end{equation*}

\begin{equation*}
Q^{*}(s,a)=max_{\pi}  r(s,a)+ \gamma \sum p(s_{t+1}=s'|s_{t}=s,a_{t}=a) Q^{\pi}(s',a')
\end{equation*}

\begin{equation*}
Q^{*}(s,a)=  r(s,a) +\gamma  max_{\pi} \sum p(s_{t+1}=s'|s_{t}=s,a_{t}=a) Q^{\pi'}(s',a')
\end{equation*} where $\pi ' =(a',\pi')$
The question is now to enter the max operator in the sum. We have the inequality that in general $max(\sum)<\sum(max)$
Then we have considering $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$ \begin{align} \sum{(s'} p(s{t+1}=s'|s_t = s, at = a ) \max{\pi'}Q^{\pi'}(s',a') &= \sum{(s')} p(s{t+1}=s' |s_t = s, at = a ) Q^{\hat{\pi}(s',a')}(s',a') \ &\leq \max{\pi'} \sum{s'} p(s{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \end{align}

So we can rewrite our initial equation as : \begin{align} Q^{}(s,a)&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, at = a )\max{a'}\max{\pi'}Q^{\pi'}(s',a') \ &= r(s,a) + \gamma \sum(p(s{t+1}=s' |s_t = s, at = a )\max{a'} Q^{}(s',a') \ &=E{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max{a'}Q^{*}(s',a')]. \end{align}

**(3)**
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

We can indeed use this objective function as it will enforce the previous equality which is necessary to make our neural network converge.



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [14]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        # the following condition helps us maintaining the max memory to 100
        if len(self.memory)>self.max_memory:
            del self.memory[0]
            
    def random_access(self):
        # index of the move we are going to access
        i = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [15]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
         # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
         # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

             # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [16]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Firstly we remember the states, then we learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # first we retrieve all elements of random access 
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            # the target_q is what our model will predict considering the values of s_
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            # we improve our prediction only if there is a new informtion
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                # /////////// INFOS INFOS INFOS////////////
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu')) # we used only one hidden layer
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [29]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
# the suggested parameters gave the best results for this architecture
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0015 | Win/lose count 1.5/1.0 (0.5)
Epoch 001/050 | Loss 0.0102 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/050 | Loss 0.0252 | Win/lose count 3.0/5.0 (-2.0)
Epoch 003/050 | Loss 0.0036 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/050 | Loss 0.0094 | Win/lose count 5.0/5.0 (0.0)
Epoch 005/050 | Loss 0.0034 | Win/lose count 1.5/0 (1.5)
Epoch 006/050 | Loss 0.0084 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/050 | Loss 0.0070 | Win/lose count 5.5/4.0 (1.5)
Epoch 008/050 | Loss 0.0052 | Win/lose count 0.5/4.0 (-3.5)
Epoch 009/050 | Loss 0.0075 | Win/lose count 3.5/10.0 (-6.5)
Epoch 010/050 | Loss 0.0042 | Win/lose count 4.0/6.0 (-2.0)
Epoch 011/050 | Loss 0.0102 | Win/lose count 5.0/4.0 (1.0)
Epoch 012/050 | Loss 0.0051 | Win/lose count 2.0/1.0 (1.0)
Epoch 013/050 | Loss 0.0296 | Win/lose count 2.0/6.0 (-4.0)
Epoch 014/050 | Loss 0.0041 | Win/lose count 4.5/7.0 (-2.5)
Epoch 015/050 | Loss 0.0051 | Win/lose count 3.0/4.0 (-1.0)
Epoch 016/050 | Loss 0.0159 | Win/lose count 7.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [17]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        # we use two convolutionnal layers here 
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [18]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0010 | Win/lose count 3.0/1.0 (2.0)
Epoch 001/050 | Loss 0.0008 | Win/lose count 5.0/3.0 (2.0)
Epoch 002/050 | Loss 0.0044 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/050 | Loss 0.0023 | Win/lose count 3.0/7.0 (-4.0)
Epoch 004/050 | Loss 0.0052 | Win/lose count 7.5/4.0 (3.5)
Epoch 005/050 | Loss 0.0028 | Win/lose count 7.5/3.0 (4.5)
Epoch 006/050 | Loss 0.0047 | Win/lose count 10.5/2.0 (8.5)
Epoch 007/050 | Loss 0.0053 | Win/lose count 6.0/2.0 (4.0)
Epoch 008/050 | Loss 0.0021 | Win/lose count 3.5/4.0 (-0.5)
Epoch 009/050 | Loss 0.1047 | Win/lose count 9.0/3.0 (6.0)
Epoch 010/050 | Loss 0.0015 | Win/lose count 11.5/4.0 (7.5)
Epoch 011/050 | Loss 0.0052 | Win/lose count 11.5/6.0 (5.5)
Epoch 012/050 | Loss 0.0026 | Win/lose count 7.0/2.0 (5.0)
Epoch 013/050 | Loss 0.0050 | Win/lose count 7.0/3.0 (4.0)
Epoch 014/050 | Loss 0.0521 | Win/lose count 8.0/2.0 (6.0)
Epoch 015/050 | Loss 0.0037 | Win/lose count 11.0/0 (11.0)
Epoch 016/050 | Loss 0.0022 | Win/lose count 13.0/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [19]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.0/1.0. Average score (6.0)
Win/lose count 6.5/4.0. Average score (4.25)
Win/lose count 18.5/1.0. Average score (8.666666666666666)
Win/lose count 21.0/5.0. Average score (10.5)
Win/lose count 6.5/3.0. Average score (9.1)
Win/lose count 12.0/2.0. Average score (9.25)
Win/lose count 7.0/0. Average score (8.928571428571429)
Win/lose count 8.0/3.0. Average score (8.4375)
Win/lose count 18.0/2.0. Average score (9.277777777777779)
Win/lose count 8.0/2.0. Average score (8.95)
Win/lose count 13.5/1.0. Average score (9.272727272727273)
Final score: 9.272727272727273
Test of the FC
Win/lose count 4.5/5.0. Average score (-0.5)
Win/lose count 4.5/5.0. Average score (-0.5)
Win/lose count 2.5/4.0. Average score (-0.8333333333333334)
Win/lose count 3.0/2.0. Average score (-0.375)
Win/lose count 3.5/7.0. Average score (-1.0)
Win/lose count 6.0/5.0. Average score (-0.6666666666666666)
Win/lose count 4.0/12.0. Average score (-1.7142857142857142)
Win/lose count 2.5/5.0. A

In [20]:
HTML(display_videos('cnn_test10.mp4'))

In [21]:
HTML(display_videos('fc_test10.mp4'))

We can see here that the final score is higher for the CNN than for the fully connected layer (9.27 compared to -1.45). We can modify the temperature to see that this tendency is still verified but for clarity I decided not to include all results for dfferent temperatures. ### A compléter en faisant d'autres tests, changer epoch train. 
The main issue we observe on the simulations is that it tends to go back to positions already taken (take a step back and do the same back a forth step over and over again). 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [22]:
def train_explore(agent,env,epoch,modif_epsilon=0.3,prefix=''):
    #This new procedure encourages exploration via a parameter modif_epsilon that decreases epsilon when exploring
    
    
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        # MODIFICATION HERE : we set the epsilon value to a percentage of the previous epsilon via the modif-epsiln parameter
        agent.set_epsilon(agent.epsilon*(1-modif_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        # we define here a malus which happens if we already visited this position
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        # the malus only happens if we are in the training phase, hence the added parameter to the function act

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
    
        ## Here we add the exploring rewards
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        # we now impose a game over if the current time is superiori to the max time
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        #The malus is 0 at the beginning
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [23]:
# Training 
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0159 | Win/lose count 6.5/21.100000000000023 (-14.600000000000023)
Epoch 001/050 | Loss 0.0139 | Win/lose count 11.0/20.200000000000024 (-9.200000000000024)
Epoch 002/050 | Loss 0.0059 | Win/lose count 4.0/20.80000000000003 (-16.80000000000003)
Epoch 003/050 | Loss 0.0094 | Win/lose count 10.0/22.30000000000003 (-12.30000000000003)
Epoch 004/050 | Loss 0.0074 | Win/lose count 5.5/19.700000000000017 (-14.200000000000017)
Epoch 005/050 | Loss 0.0048 | Win/lose count 7.0/17.999999999999986 (-10.999999999999986)
Epoch 006/050 | Loss 0.0046 | Win/lose count 10.0/17.39999999999998 (-7.399999999999981)
Epoch 007/050 | Loss 0.0074 | Win/lose count 10.5/16.89999999999997 (-6.39999999999997)
Epoch 008/050 | Loss 0.0066 | Win/lose count 9.5/16.899999999999974 (-7.399999999999974)
Epoch 009/050 | Loss 0.0175 | Win/lose count 17.0/14.599999999999968 (2.4000000000000323)
Epoch 010/050 | Loss 0.0141 | Win/lose count 12.5/14.499999999999968 (-1.999999999999968)
Epoch 011/050 | L

In [24]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 18.0/5.0. Average score (13.0)
Win/lose count 33.5/2.0. Average score (22.25)
Win/lose count 12.5/3.0. Average score (18.0)
Win/lose count 25.0/5.0. Average score (18.5)
Win/lose count 23.5/3.0. Average score (18.9)
Win/lose count 17.5/3.0. Average score (18.166666666666668)
Win/lose count 23.5/0. Average score (18.928571428571427)
Win/lose count 26.0/5.0. Average score (19.1875)
Win/lose count 23.0/6.0. Average score (18.944444444444443)
Win/lose count 24.0/5.0. Average score (18.95)
Win/lose count 21.5/3.0. Average score (18.90909090909091)
Final score: 18.90909090909091


We can see now that on the videos, the rat explores more the space and doesn't seem to go back and forth on the same positions as it used to. The final score is also higher in this configuration with the same parameters as before (temperature=0.3, 50 epochs, using the DQN_CNN approach).

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***